In [1]:
import requests
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pandas as pd


ALCHEMY_POLYGON = 'https://polygon-mainnet.g.alchemyapi.io/v2/'
ALCHEMY_KEY = 'Your API key'  # Go to alchemy.com -> create an account and API key
alchemy_polygon_url = ALCHEMY_POLYGON + ALCHEMY_KEY
w3_polygon = Web3(Web3.HTTPProvider(alchemy_polygon_url))
w3_polygon.middleware_onion.inject(geth_poa_middleware, layer=0)

abi_url = 'https://raw.githubusercontent.com/cigeza/proofivy/main/proofivy_abi.json'
abi_proofivy = requests.get(abi_url).json()
proofivy_address = '0xAD67cB5Cd9c8839E4f485322bD9585d652D60DA1'
old_proofivy_address = '0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5'
contract_proofivy = w3_polygon.eth.contract(address=proofivy_address, abi=abi_proofivy)


def parse_log(contract, event, from_block=33_017_050):
    logs = getattr(contract.events, event).createFilter(fromBlock=from_block).get_all_entries()
    if len(logs) == 0:
        return pd.DataFrame()
    else:
        df = pd.DataFrame(logs)
        for key in df['args'].iloc[0].keys():
            df[key] = df['args'].apply(lambda l: l[key])
        df = df.drop(['args'], axis=1)
        return df

    
def get_timestamp_from_block_number(block_number):
    return pd.to_datetime(w3_polygon.eth.get_block(block_number)['timestamp'], utc=True, unit='s')
    

## Events

All events:
 - GuildMemberCommit
 - GuildFounded
 - GuildMemberCommit
 - GuildMemberMessage
 - PublicCommit
 - PublicMessage

In [2]:
df_guild_member_commits = parse_log(contract_proofivy, 'GuildMemberCommit')
df_guild_member_commits['block_timestamp'] = df_guild_member_commits['blockNumber'].apply(get_timestamp_from_block_number)
df_guild_member_commits

,address,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex,sender,guild,hash,guild_commit_count,block_timestamp
0,0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5,"b""_\x8b\xc6\xa6b5\xc1,\x93t\xa3\xae\xe6\x08\xe...",34004569,GuildMemberCommit,136,b'\xeb\xcc/)\xbf@j\x99\x1c\xc4\xb8\x8b$\x01%\x...,39,0x2364B7FDA95E8EBe695158E57D8AEb39351AC758,gc_lab,QmcAvT6PBh7WCGdLJ9d654KwSbER7xFUnX1SBPJd1F2ibA,1,2022-10-06 13:23:52+00:00
1,0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5,b'\xbe\xae\xfd|(\x12\xac\xc7\xb2\x0b\xf7\xe3]J...,34008771,GuildMemberCommit,178,b'\x9b\x11{\x8f\x01\xb3b\xe3UC\xd1\x89\x8e{\x1...,45,0x2364B7FDA95E8EBe695158E57D8AEb39351AC758,gc_lab,QmPHJ9KtP9gUyMMuX8yPqVZie2oLDAmnfYhQTYFTXntmnw,2,2022-10-06 15:48:28+00:00
2,0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5,b'\t\xb0\xc5\xe1N\xd1\xd1\t\x95\xd3L\xfe(\xa7\...,34085502,GuildMemberCommit,191,"b""\xb5}\xd4)C\xff\xa2\x1f\nU'7F|\xda$G\xd4\x9f...",45,0x2364B7FDA95E8EBe695158E57D8AEb39351AC758,gc_lab,QmYXWP3GwRr7irszj1xgXzJhXQZbkDZdiUW5JGF6DkxKiZ,3,2022-10-08 12:23:10+00:00


### Add timestamp (UTC)

In [3]:
df_guild_member_commits['block_timestamp'] = df_guild_member_commits['blockNumber'].apply(get_timestamp_from_block_number)
df_guild_member_commits

,address,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex,sender,guild,hash,guild_commit_count,block_timestamp
0,0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5,"b""_\x8b\xc6\xa6b5\xc1,\x93t\xa3\xae\xe6\x08\xe...",34004569,GuildMemberCommit,136,b'\xeb\xcc/)\xbf@j\x99\x1c\xc4\xb8\x8b$\x01%\x...,39,0x2364B7FDA95E8EBe695158E57D8AEb39351AC758,gc_lab,QmcAvT6PBh7WCGdLJ9d654KwSbER7xFUnX1SBPJd1F2ibA,1,2022-10-06 13:23:52+00:00
1,0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5,b'\xbe\xae\xfd|(\x12\xac\xc7\xb2\x0b\xf7\xe3]J...,34008771,GuildMemberCommit,178,b'\x9b\x11{\x8f\x01\xb3b\xe3UC\xd1\x89\x8e{\x1...,45,0x2364B7FDA95E8EBe695158E57D8AEb39351AC758,gc_lab,QmPHJ9KtP9gUyMMuX8yPqVZie2oLDAmnfYhQTYFTXntmnw,2,2022-10-06 15:48:28+00:00
2,0xCc26e45546DCb8D747C2bB69e348F0515EF3eFc5,b'\t\xb0\xc5\xe1N\xd1\xd1\t\x95\xd3L\xfe(\xa7\...,34085502,GuildMemberCommit,191,"b""\xb5}\xd4)C\xff\xa2\x1f\nU'7F|\xda$G\xd4\x9f...",45,0x2364B7FDA95E8EBe695158E57D8AEb39351AC758,gc_lab,QmYXWP3GwRr7irszj1xgXzJhXQZbkDZdiUW5JGF6DkxKiZ,3,2022-10-08 12:23:10+00:00


## Call info from contract state

In [4]:
contract_proofivy.all_functions()

[<Function change_owner(address)>,
 <Function set_public_commit_price(uint256)>,
 <Function set_public_message_price(uint256)>,
 <Function public_commit(string)>,
 <Function public_message(string)>,
 <Function found_guild(string,address)>,
 <Function add_admin(string,address)>,
 <Function remove_admin(string,address)>,
 <Function aspire_membership(string)>,
 <Function remove_aspiring_membership(string)>,
 <Function add_member(string,address)>,
 <Function remove_member(string,address)>,
 <Function guild_commit(string,string)>,
 <Function guild_message(string,string)>,
 <Function public_commit_price()>,
 <Function public_message_price()>,
 <Function public_commit_counter()>,
 <Function public_commit_senders(uint256)>,
 <Function public_commits(uint256)>,
 <Function public_message_counter()>,
 <Function public_message_senders(uint256)>,
 <Function public_messages(uint256)>,
 <Function guilds(string)>,
 <Function guild_admins(string,address)>,
 <Function guild_aspiring_members(string,addre

In [5]:
contract_proofivy.functions.guild_commits('gc_lab', 1).call()

'QmcAvT6PBh7WCGdLJ9d654KwSbER7xFUnX1SBPJd1F2ibA'